**Maximize Profit for a Glass Manufacturer**

In this exercise you are planning the production at a glass manufacturer. This manufacturer only produces wine and beer glasses:

- there is a maximum production capacity of 60 hours
- each batch of wine and beer glasses takes 6 and 5 hours respectively
- the warehouse has a maximum capacity of 150 rack spaces
- each batch of the wine and beer glasses takes 10 and 20 spaces respectively
- the production equipment can only make full batches, no partial batches

Also, we only have orders for 6 batches of wine glasses. Therefore, we do not want to produce more than this. Each batch of the wine glasses earns a profit of \\$5 and the beer \\$4.5.

The objective is to maximize the profit for the manufacturer.

In [1]:
# importing pulp
from pulp import *

In [2]:
# Initialize Class
model = LpProblem('glass_mfg', sense=LpMaximize)
# decision variables
W = LpVariable('W', lowBound=0, upBound=None, cat='Integer')
B = LpVariable('B', lowBound=0, upBound=None, cat='Integer')
# objective funcrion
model += 5*W + 4.5*B
# constraints
model += 6*W + 5*B <= 60
model += 10*W + 20*B <= 150
model += W <= 6
# solve the model
model.solve()
print('Make {} batches of wine'.format(W.varValue))
print('Make {} batches of beer'.format(B.varValue))

Make 6.0 batches of wine
Make 4.0 batches of beer


**What if too many variables?**

- `lpSum()` to the rescue
- uses Python list comprehension

Example:

```Python
# Define Objective Function
model += 20*A + 40*B + 33*C + 14*D + 6*E + 60*F

Equivalent to:

# Define Objective Function
var_list = [20*A, 40*B, 33*C, 14*D, 6*E, 60*F]
model += lpSum(var_list)

# Define Objective Function
cake_types = ["A", "B", "C", "D", "E", "F"]
profit_by_cake = {"A":20, "B":40, "C":33, "D":14, "E":6, "F":60}
var_dict = {"A":A, "B":B, "C":C, "D":D, "E":E,"F":F}
model += lpSum([profit_by_cake[type] * var_dict[type]
                for type in cake_types])
```

**Note**:

Variable creation is a little different for a large scale probmlem with many variables. Need to use `LpVariable(name, lowBound=None, upBound=None, cat='Continuous', e=None)` to create variables and assign their categories (i.e. integer/continuous).

- `name` = The prex to the name of each LP variable created
- `lowBound` = Lower bound
- `upBound` = Upper bound
- `cat` = The type of variable this is
    - Integer
    - Binary
    - Continuous (default)

**Logistics planning problem**

You are consulting for kitchen oven manufacturer helping to plan their logistics for next month. There are two warehouse locations (New York, and Atlanta), and four regional customer locations (East, South, Midwest, West). The expected demand next month for East it is 1,800, for South it is 1,200, for the Midwest it is 1,100, and for West it is 1000. The cost for shipping each of the warehouse locations to the regional customer's is listed in the table below. Your goal is to fulfill the regional demand at the lowest price.

|Customer|New York|Atlanta
|--------|--------|-------
|East	|\\$211	|\\$232
|South	|\\$232	|\\$212
|Midwest|\\$240	|\\$230
|West	|\\$300	|\\$280

Solution:

In [3]:
# Initialize Model
model = LpProblem("MinimizeTransportationCosts", LpMinimize)

# build costs dictionary
costs = {('Atlanta', 'East'): 232, 
         ('Atlanta', 'Midwest'): 230, 
         ('Atlanta', 'South'): 212, 
         ('Atlanta', 'West'): 280, 
         ('New York', 'East'): 211, 
         ('New York', 'Midwest'): 240, 
         ('New York', 'South'): 232, 
         ('New York', 'West'): 300}

# Build the lists and the demand dictionary
warehouse = ['New York', 'Atlanta']
customers = ['East', 'South', 'Midwest', 'West']
regional_demand = [1800, 1200, 1100, 1000]
demand = dict(zip(customers, regional_demand))

# define decision variables by first using list comprehension to 
# iterate over the warehouse, and customers lists to create a list of keys.
# Use that list of keys with LpVariable.dicts() to define the variables needed.
key = [(w, c) for w in warehouse for c in customers]
var_dict = LpVariable.dicts('shipments', 
                            key, 
                            lowBound = 0, 
                            upBound=None, 
                            cat='Integer')

# Define Objective
model += lpSum([costs[(w, c)] * var_dict[(w, c)] 
                for c in customers for w in warehouse])

# For each customer, sum warehouse shipments and set equal to customer demand
for c in customers:
    model += lpSum([var_dict[(w, c)] for w in warehouse]) == demand[c]

model.solve()

1

In [4]:
# variables values
for v in model.variables():
    print(v.name, "=", v.varValue)

shipments_('Atlanta',_'East') = 0.0
shipments_('Atlanta',_'Midwest') = 1100.0
shipments_('Atlanta',_'South') = 1200.0
shipments_('Atlanta',_'West') = 1000.0
shipments_('New_York',_'East') = 1800.0
shipments_('New_York',_'Midwest') = 0.0
shipments_('New_York',_'South') = 0.0
shipments_('New_York',_'West') = 0.0


**Run cell below for more info about the method `LpVariable.dicts()`**

In [5]:
LpVariable.dicts?